In [1]:
! pip install -r requirements.txt

In [2]:
import requests
from bs4 import BeautifulSoup
import time

In [3]:
event_list = []

for i in range(1, 42):
    url = "https://visitseattle.org/events/page/"+str(i)

    # time.sleep(1) #wait for 1 second
    res = requests.get(url) #read information from the server

    # check if the server response is ok
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "html.parser")
        selector = "div.search-result-preview > div > h3 > a"
        a_eles = soup.select(selector)

        #add lists of events to event_list
        event_list += [x['href'] for x in a_eles]

print (event_list)


['https://visitseattle.org/events/tribute-to-john-williams/', 'https://visitseattle.org/events/lakecia-benjamin-and-phoenix/', 'https://visitseattle.org/events/corey-harper/', 'https://visitseattle.org/events/dj-shadow/', 'https://visitseattle.org/events/opera-on-tap/', 'https://visitseattle.org/events/pbs-peter-band-supreme/', 'https://visitseattle.org/events/seattle-kraken-vs-chicago-blackhawks-2/', 'https://visitseattle.org/events/silent-planet/', 'https://visitseattle.org/events/stuff-you-should-know/', 'https://visitseattle.org/events/the-cooking-gene/', 'https://visitseattle.org/events/aryana-leon/', 'https://visitseattle.org/events/colorworks/', 'https://visitseattle.org/events/dar-williams/', 'https://visitseattle.org/events/deb-perelman/', 'https://visitseattle.org/events/emanuel-brown/', 'https://visitseattle.org/events/james-miles/', 'https://visitseattle.org/events/madeline-pendleton/', 'https://visitseattle.org/events/october-london/', 'https://visitseattle.org/events/taco

In [4]:
import csv


In [5]:
# open a csv file to write
with open('event_list.csv', 'w', newline = '') as f:
    #create a csv writer object
    writer = csv.writer(f)

    # write the header
    writer.writerow(["Name", "Date & Time", "Location", "Type", "Region"])

    # loop through each event and write data to csv file
    for event_url in event_list:
        res = requests.get(event_url)

        # check if the server response is ok
        if res.status_code == 200:
            event_info = []
            # parse the html text
            soup = BeautifulSoup(res.text, "html.parser")

            # get the name of the event
            name_selector = "div.medium-6.columns.event-top > h1"
            name_eles = soup.select(name_selector)
            event_info.append(name_eles[0].text)
            
            # get the date and time of the event
            date_and_time_selector = "div.medium-6.columns.event-top > h4 > span:nth-child(1)"
            date_and_time_eles = soup.select(date_and_time_selector)
            event_info.append(date_and_time_eles[0].text)

            # get the location of the event
            location_selector = "div.medium-6.columns.event-top > h4 > span:nth-child(2)"
            location_eles = soup.select(location_selector)
            event_info.append(location_eles[0].text)

            # get the type of the event
            type_selector = "div.medium-6.columns.event-top > a:nth-child(3)"
            type_eles = soup.select(type_selector)
            event_info.append(type_eles[0].text)

            # get the region of the event
            region_selector = "div.medium-6.columns.event-top > a:nth-child(4)"
            region_eles = soup.select(region_selector)
            event_info.append(region_eles[0].text)

            writer.writerow(event_info)

                            

In [6]:
import requests
import datetime
import re

In [7]:
# read the event file into a list of event dictionaries, using the header row to determine column names.
events_file = open('event_list.csv', 'r')
events = list( csv.DictReader(events_file, delimiter=','))
events_file.close()

with open('event_weather_info.csv', 'w', newline = '') as f:
    # create a csv writer object
    writer = csv.writer(f)

    # write the header
    writer.writerow(["Name", "Region", "Weathers"])
    

    # look up the weather for the region of each event
    for event in events:

        date = event['Date & Time']

        if re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', date): # check if the date is in the format of mm/dd/yyyy

            # get the coordinate of the event by using the region
            region = event['Region']
            res = requests.get(f"https://nominatim.openstreetmap.org/search.php?q={region}&format=jsonv2")
            region = res.json()
            if res.status_code == 200 and region != []:
                
                # get the latitude and longitude of the region
                lat, lon = region[0]['lat'], region[0]['lon']

                # get the weather for the location using the latitude and longitude
                res = requests.get(f"https://api.weather.gov/points/{lat},{lon}")
                weather_point = res.json()
                if res.status_code == 200:
        
                    # get the weather forecast
                    forecast_url = weather_point['properties']['forecast']
                    res = requests.get(forecast_url)
                    forecast = res.json()

                    # get the weather for the date of the event
                    for period in forecast['properties']['periods']:
                        if period['isDaytime']:
                            start_time = datetime.datetime.fromisoformat(period['startTime']).date()
                            if start_time == datetime.datetime.strptime(date,"%m/%d/%Y").date():
                                temperature = period['temperature']
                                short_forecast = period['shortForecast']
                                weather = f"{temperature}°F, {short_forecast}" # will show in the format of (70°F, Sunny)
                                print([event['Name'], event['Date & Time'],event['Region'], weather])
                                writer.writerow([event['Name'], event['Date & Time'], event['Region'], weather])
                                break


['Corey Harper', '1/24/2024', 'Capitol Hill / Central District', '48°F, Rain']
['DJ Shadow', '1/24/2024', 'University District', '50°F, Light Rain']
['Opera on Tap', '1/24/2024', 'South Seattle', '49°F, Rain']
['PBS – Peter Band Supreme', '1/24/2024', 'Wallingford / Greenlake', '48°F, Rain']
['Seattle Kraken vs. Chicago Blackhawks', '1/24/2024', 'Queen Anne / Seattle Center', '49°F, Rain']
['Silent Planet', '1/24/2024', 'Downtown', '62°F, Partly Sunny']
['Stuff You Should Know', '1/24/2024', 'Downtown', '62°F, Partly Sunny']
['The Cooking Gene', '1/24/2024', 'Downtown', '62°F, Partly Sunny']
['Aryana Leon', '1/25/2024', 'Capitol Hill / Central District', '49°F, Light Rain']
['Colorworks', '1/25/2024', 'Fremont / Ballard', '60°F, Areas Of Fog then Mostly Sunny']
['Dar Williams', '1/25/2024', 'University District', '50°F, Light Rain']
['Deb Perelman', '1/25/2024', 'Downtown', '64°F, Patchy Fog then Partly Sunny']
['Emanuel Brown', '1/25/2024', 'Capitol Hill / Central District', '49°F, Li